In [1]:
# Copyright 2025 Forusone

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Agent Development Kit - Session
* [Memory: Long-Term Knowledge with MemoryService](https://google.github.io/adk-docs/sessions/memory/#how-memory-works-in-practice)

### Install adk

In [2]:
%pip install --upgrade --quiet google-adk

In [3]:
# @title Project Information

PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"

MODEL_LIST = ["gemini-2.0-flash", \
              "gemini-2.5-pro-preview-03-25", \
              "gemini-2.5-flash-preview-04-17"]

MODEL_NAME = MODEL_LIST[2]
print(MODEL_NAME)

gemini-2.5-flash-preview-04-17


In [4]:
# @title Authentication to access to GCP
import sys
from pprint import pprint

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id = PROJECT_ID)

### Initialize Vertex AI

* Please refer to configuration to **initialize Vertex AI : [Link](https://google.github.io/adk-docs/get-started/quickstart/#set-up-the-model)**
* You can set up api key rather than usning Vertex AI.

In [5]:
import os

GOOGLE_GENAI_USE_VERTEXAI = "TRUE"                            #@param {type:"string"}
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = GOOGLE_GENAI_USE_VERTEXAI

GOOGLE_CLOUD_PROJECT = "ai-hangsik"                           #@param {type:"string"}
os.environ["GOOGLE_CLOUD_PROJECT"] = GOOGLE_CLOUD_PROJECT

GOOGLE_CLOUD_LOCATION = "us-central1"                         #@param {type:"string"}
os.environ["GOOGLE_CLOUD_LOCATION"] = GOOGLE_CLOUD_LOCATION


## Session store

In [6]:
import sqlite3
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import InMemorySessionService, Session

## Store information into Memory

In [7]:
#@title Build a Agent with adk.
from google.adk.agents import LlmAgent
from google.adk.tools import google_search
from google.genai import types
from google.adk.tools import load_memory # Tool to query memory
from google.adk.memory import InMemoryMemoryService # Import MemoryService
from google.adk.runners import Runner


PROMPT = """
You are a helpful assistant. Answer user questions using Google Search when needed.
Wirte a simple and clear answer to the user question.
"""

memory_insert_agent = LlmAgent(
      name="Memory_insert_agent",
      model=MODEL_NAME,
      instruction=PROMPT,
      description="An assistant that can search the web.",
      tools=[google_search] # Use Google search tool
)

#------------------------------------------------------------------------------------

PROMPT = """
Answer the user's question. Use the 'load_memory' tool "
 "if the answer might be in past conversations.
"""

memory_recall_agent = LlmAgent(
      name="Memory_recall_agent",
      model=MODEL_NAME,
      instruction=PROMPT,
      description="Recall memory",
      tools=[load_memory] )



## Session

In [8]:
APP_NAME = "Search_Assistant"
USER_ID = "shins"

memory_insert_service = InMemorySessionService()
memory_insert_session = memory_insert_service.create_session(app_name=APP_NAME, user_id=USER_ID)

memory_recall_service = InMemoryMemoryService()

In [9]:
runner = Runner(
    agent=memory_insert_agent,
    app_name=APP_NAME,
    session_service=memory_insert_service,
    memory_service=memory_recall_service  # memory_service 에 InMemoryMemoryService 를 등록
)

In [10]:
query = "What is the exchange rate from US dollars to Korean currency recently ?"

content = types.Content(role='user', parts=[types.Part(text=query)])
events = runner.run(user_id=memory_insert_session.user_id,
                    session_id=memory_insert_session.id,
                    new_message=content)

for event in events:
    if event.is_final_response():
        final_response = event.content.parts[0].text
        pprint(f"Agent Response: {final_response}\n")
    else:
        pprint("No results")


('Agent Response: As of recently, the exchange rate from US dollars to Korean '
 'currency (South Korean Won) is approximately 1 USD = 1437-1439 KRW.\n'
 '\n'
 'Please note that exchange rates fluctuate constantly, so the exact rate may '
 'vary slightly depending on the time and the specific service used for '
 'conversion. Sources show rates like 1439.4050 KRW, 1437.42 KRW, and '
 '1438.363268 KRW for 1 USD. The 30-day average has been around 1443.6920 '
 'KRW.\n')


In [11]:
completed_session = memory_insert_service.get_session(app_name=memory_insert_session.app_name, user_id=memory_insert_session.user_id, session_id=memory_insert_session.id)
print(completed_session)

id='9d267806-a809-4876-905a-6e4be4a0c1bf' app_name='Search_Assistant' user_id='shins' state={} events=[Event(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='What is the exchange rate from US dollars to Korean currency recently ?')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, invocation_id='e-8a4e1dda-2ba6-47cf-8cb3-8c38ba847d07', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='d1F1DAXF', timestamp=1745752033.542265), Event(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function

In [12]:
#@title add information into memory service.
memory_recall_service.add_session_to_memory(completed_session)


In [13]:
#@title Set agent with Memory recall agent
runner.agent = memory_recall_agent

In [14]:
memory_insert_session2 = memory_insert_service.create_session(app_name=memory_insert_session.app_name, user_id=memory_insert_session.user_id)

In [15]:
from google.adk.runners import Runner

query = "What is the exchange rate which I've retrieved ?"
content = types.Content(role='user', parts=[types.Part(text=query)])

events = runner.run(user_id=memory_insert_session2.user_id,
                    session_id=memory_insert_session2.id,
                    new_message=content)

for event in events:
    if event.is_final_response():
        final_response = event.content.parts[0].text
        pprint(f"Agent Response: {final_response}\n")
    else:
        pprint("No results")

'No results'
'No results'
('Agent Response: Based on our previous conversation, the exchange rate from '
 'US dollars to Korean currency (South Korean Won) was approximately 1 USD = '
 '1437-1439 KRW. Sources showed rates like 1439.4050 KRW, 1437.42 KRW, and '
 '1438.363268 KRW for 1 USD. The 30-day average was around 1443.6920 KRW.\n')


## End of notebook